## Creación del dataset


In [1]:
import os
import json
import random
import pandas as pd
import ast 
import locale
from collections import Counter
from datetime import datetime

In [2]:
# Establecer el locale a español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

El método elegido para la extracción de los datos es un modelo BERT para Named Entity Recognition (NER), así que el dataset tiene que tener también etiquetas NER, que serán generadas posteriormente.

In [3]:
#Renombrar los textos extraidos

# Definir la ruta de la carpeta
carpeta = "Data/training_texto"

# Obtener la lista de archivos en la carpeta
archivos = os.listdir(carpeta)

# Iterar sobre cada archivo en la carpeta
for archivo in archivos:
    # Verificar si el archivo tiene la extensión .pdf.txt
    if archivo.endswith('.pdf.txt'):
        # Generar el nuevo nombre de archivo eliminando la parte .pdf
        nuevo_nombre = archivo.replace('.pdf', '')

        # Ruta completa del archivo original y nuevo
        ruta_original = os.path.join(carpeta, archivo)
        ruta_nueva = os.path.join(carpeta, nuevo_nombre)

        # Renombrar el archivo
        os.rename(ruta_original, ruta_nueva)


In [4]:
# Definir las rutas de las carpetas
txt_folder = "Data/training_texto"
json_folder = "Data/training"

# Función para leer el contenido de un archivo de texto
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Lista para almacenar los datos de cada factura
data = []

# Iterar sobre los archivos de texto
for txt_file in os.listdir(txt_folder):
    if txt_file.endswith(".txt"):
        txt_path = os.path.join(txt_folder, txt_file)
        json_file = txt_file.replace(".txt", ".json")
        json_path = os.path.join(json_folder, json_file)
        
        # Leer el contenido del archivo de texto
        text_content = read_txt_file(txt_path)
        
        # Leer el contenido del archivo JSON
        with open(json_path, 'r', encoding='utf-8') as json_file:
            json_data = json.load(json_file)
        
        # Obtener el ID del nombre del archivo
        file_id = os.path.splitext(txt_file)[0]
        
        # Agregar los datos al listado
        data.append({
            'id': file_id,
            'texto': text_content,
            'ner_tags': [],  
            'json': json_data
        })

# Convertir la lista de datos a un DataFrame de pandas
df = pd.DataFrame(data)

Eliminamos columnas en las que coinciden nombres de poblaciones y provincias, ya que dificultan el etiquetado y pueden influir negativamente a la hora de entrenar el modelo.








In [5]:
def contar_y_eliminar_coincidencias(df):
    # Inicializamos los contadores
    count_poblacion_prov = 0
    count_poblacion_prov_comer_prov_comer = 0
    count_poblacion_poblacion_prov = 0
    count_cp_cp_comer = 0
    count_prov_prov_comer = 0

    # Lista para almacenar el recuento de filas donde población_comercializadora es igual a provincia_comercializadora
    poblacion_prov_comer = []
    facturas_eliminadas = []

    # Iteramos sobre cada fila del DataFrame
    for index, row in df.iterrows():
        json_data = row['json']

        # Verificamos si los campos son iguales y actualizamos los contadores
        if json_data['población_cliente'] == json_data['provincia_cliente']:
            count_poblacion_prov += 1

        if json_data['población_comercializadora'] == json_data['provincia_comercializadora']:
            count_poblacion_prov_comer_prov_comer += 1
            poblacion_prov_comer.append(index)  # Añadimos el índice de la fila a la lista
            facturas_eliminadas.append(row['id'])  # Añadimos el id de la factura a la lista de eliminadas

        if json_data['población_cliente'] == json_data['población_comercializadora']:
            count_poblacion_poblacion_prov += 1

        if json_data['cp_cliente'] == json_data['cp_comercializadora']:
            count_cp_cp_comer += 1

        if json_data['provincia_cliente'] == json_data['provincia_comercializadora']:
            count_prov_prov_comer += 1

    # Eliminar las filas del DataFrame original
    df.drop(poblacion_prov_comer, inplace=True)
    
    # Resetear el índice del DataFrame
    df.reset_index(drop=True, inplace=True)

    # Imprimimos los resultados
    print("Recuento de filas donde población_cliente es igual a provincia_cliente:", count_poblacion_prov)
    print("Recuento de filas donde población_comercializadora es igual a provincia_comercializadora:", count_poblacion_prov_comer_prov_comer)
    print("Recuento de filas donde población_cliente es igual a población_comercializadora:", count_poblacion_poblacion_prov)
    print("Recuento de filas donde cp_cliente es igual a cp_comercializadora:", count_cp_cp_comer)
    print("Recuento de filas donde provincia_cliente es igual a provincia_comercializadora:", count_prov_prov_comer)

    return facturas_eliminadas

# Llamada a la función con el DataFrame 'df'
facturas_eliminadas = contar_y_eliminar_coincidencias(df)




Recuento de filas donde población_cliente es igual a provincia_cliente: 42
Recuento de filas donde población_comercializadora es igual a provincia_comercializadora: 86
Recuento de filas donde población_cliente es igual a población_comercializadora: 0
Recuento de filas donde cp_cliente es igual a cp_comercializadora: 0
Recuento de filas donde provincia_cliente es igual a provincia_comercializadora: 8


Para mejorar la capacidad del modelo, se aumentan los datos de entrenamiento. El método elegido es separar el texto en lotes de párrafos y reordenarlos aleatoriamente. Se añade al dataset una versión reordenada de cada factura, doblando así su tamaño.








In [6]:
def augment_data(text):
    def extract_paragraphs(text):
        # Separar el texto en párrafos usando el salto de línea como delimitador
        paragraphs = text.split('\n')
        # Eliminar los párrafos vacíos
        paragraphs = [para.strip() for para in paragraphs if para.strip()]
        return paragraphs

    # Dividir el texto en párrafos
    paragraphs = extract_paragraphs(text)
    # Mezclar los párrafos en lotes de tamaño aleatorio
    batched_paragraphs = []
    current_batch = []
    for para in paragraphs:
        current_batch.append(para)
        # Aleatoriamente decidir si empezar un nuevo lote
        if len(current_batch) == random.randint(1, 5):
            # Mezclar los párrafos en el lote actual
            random.shuffle(current_batch)
            # Agregar el lote a la lista de lotes
            batched_paragraphs.append(current_batch)
            # Reiniciar el lote actual
            current_batch = []
    # Si hay párrafos restantes, agregarlos al último lote
    if current_batch:
        random.shuffle(current_batch)
        batched_paragraphs.append(current_batch)
    # Mezclar aleatoriamente los lotes
    random.shuffle(batched_paragraphs)
    # Unir los lotes en un solo texto
    augmented_text = '\n'.join(['\n'.join(batch) for batch in batched_paragraphs])
    return augmented_text




In [7]:

# Aplicar la función augment_data a la columna "texto"
df['texto_aumentado'] = df['texto'].apply(augment_data)

# Crear la columna "original" con valores 1 para las filas originales y 0 para las duplicadas
df['original'] = 1

# Crear un duplicado del DataFrame con las filas duplicadas
df_duplicated = df.copy()

# Cambiar los valores de la columna "original" en el DataFrame duplicado a 0
df_duplicated['original'] = 0

# Modificar la columna "id" según lo solicitado
df_duplicated['id'] = df_duplicated['id'] + '_sh'

# Concatenar el DataFrame original con el DataFrame duplicado
df = pd.concat([df, df_duplicated], ignore_index=True)

# Utilizando apply con una función lambda para actualizar la columna "texto" en función de la columna "original"
df['texto'] = df.apply(lambda row: row['texto_aumentado'] if not row['original'] else row['texto'], axis=1)

# Eliminar la columna "texto_transformado"
df.drop(columns=['texto_aumentado'], inplace=True)

# Aplicar split() a la columna "texto"
df['texto'] = df['texto'].apply(lambda x: x.split())

# Mostrar el DataFrame final
df

,id,texto,ner_tags,json,original
0,factura_836,"[DATOS, DE, LA, FACTURA, IMPORTE, FACTURA:, 21...",[],"{'nombre_cliente': 'ERMINIA CUESTA MACHADO', '...",1
1,factura_264,"[L, ..., u, .., n, ..e, .., s, .., ., a, .., ....",[],"{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...",1
2,factura_533,"[DATOS, DE, LA, FACTURA, Nº, factura:, I336000...",[],"{'nombre_cliente': 'Iluminada Donato Vallejo',...",1
3,factura_311,"[L, ..., u, .., n, ..e, .., s, .., ., a, .., ....",[],{'nombre_cliente': 'Celestino Estrada Santiago...,1
4,factura_473,"[L, ..., u, .., n, ..e, .., s, .., ., a, .., ....",[],"{'nombre_cliente': 'Candelaria Ayala Icaza', '...",1
...,...,...,...,...,...
1823,factura_156_sh,"[X,XX, €, DETALLE, DE, LA, FACTURA, 35002, Las...",[],"{'nombre_cliente': 'SATURIO LEYVA TAMES', 'dni...",0
1824,factura_97_sh,"[automática), porcentajes, del, promedio, naci...",[],"{'nombre_cliente': 'Fermín Vívez Padilla', 'dn...",0
1825,factura_501_sh,"[DATOS, DE, LA, FACTURA, 14546, Santaella, CIF...",[],"{'nombre_cliente': 'EUSTAQUIO ARROLLO NIETO', ...",0
1826,factura_731_sh,"[Fuente:, CNMC, (Comisión, Nacional, de, los, ...",[],"{'nombre_cliente': 'CIRIACO LIRA PEÑA', 'dni_c...",0


In [8]:
def separar_puntuacion(lista_palabras):
    palabras_nuevas = []
    for palabra in lista_palabras:
        # Verificar si la palabra comienza con un paréntesis de apertura
        if palabra.startswith("(") and len(palabra) > 1:
            palabras_nuevas.append(palabra[0])  # Agregar el paréntesis de apertura
            palabra = palabra[1:]  # Remover el paréntesis de apertura para las siguientes verificaciones

        # Verificar si la palabra termina con puntuación o paréntesis de cierre
        if palabra.endswith((".", ",", ";", ":", ")", "?")) and len(palabra) > 1:
            palabras_nuevas.append(palabra[:-1])
            palabras_nuevas.append(palabra[-1])
        else:
            palabras_nuevas.append(palabra)
    return palabras_nuevas



In [9]:
def unir_puntuacion(lista_palabras):
    palabras_originales = []
    buffer = ""
    
    for palabra in lista_palabras:
        if palabra in ("(", ".", ",", ";", ":", ")", "?"):
            buffer += palabra
        else:
            if buffer:
                palabras_originales[-1] += buffer
                buffer = ""
            palabras_originales.append(palabra)
    
    if buffer:
        palabras_originales[-1] += buffer
    
    return palabras_originales


In [10]:
df['texto'] = df['texto'].apply(separar_puntuacion)


El primer paso para crear las etiquetas NER es inicializar una lista con una etiqueta 'O' para cada palabra del texto. Esta etiqueta no tendrá valor para el modelo.








In [11]:
# Función para inicializar las etiquetas NER
def initialize_ner_tags(row):
    num_words = len(row['texto'])
    return ['O'] * num_words

# Aplicar la función a cada fila del DataFrame
df['ner_tags'] = df.apply(initialize_ner_tags, axis=1)



In [12]:
# Agregar una columna 'json_comprobacion' con diccionario vacío
df['json_comprobacion'] = [{} for _ in range(len(df))]

Se definen las etiquetas NER: dos por cada clave del diccionario (B- para la primera palabra, I- para las siguientes).








In [13]:
label_list = ['O', 'B-NOM', 'I-NOM', 'B-DNI', 'I-DNI', 'B-CAL', 'I-CAL', 'B-CP', 'I-CP', 'B-LOC', 'I-LOC', 'B-PRO', 'I-PRO', 
            'B-NOMC', 'I-NOMC', 'B-CIF', 'I-CIF', 'B-DIRC', 'I-DIRC', 'B-CPC', 'I-CPC', 'B-LOCC', 'I-LOCC', 'B-PROC', 'I-PROC', 
            'B-ŃUMF', 'I-NUMF', 'B-INI', 'I-INI', 'B-FIN', 'I-FIN', 'B-FAC', 'I-FAC', 'B-CAR','I-CAR', 'B-PER',
            'I-PER', 'B-POT', 'I-POT']
              

Esta función itera por los diccionarios de la columna 'json', extrayendo el valor de cada clave para luego buscar esa secuencia en la columna 'texto' y usar los índices de esa secuencia para crear las etiquetas NER en 'ner_tags'.








In [14]:
def crear_etiquetas(df):
    for index, row in df.iterrows():
        # Paso 1: Extraer el nombre del cliente y convertirlo en lista de palabras
        nombre_cliente = row['json']['nombre_cliente']
        palabras_nombre_cliente = nombre_cliente.split()
        palabras_nombre_cliente = separar_puntuacion(palabras_nombre_cliente)
        
        # Paso 1.1: Extraer el DNI del cliente y convertirlo en lista de palabras
        dni_cliente = row['json']['dni_cliente']
        palabras_dni_cliente = dni_cliente.split()
        palabras_dni_cliente = separar_puntuacion(palabras_dni_cliente)
        
        # Paso 1.2: Extraer la calle del cliente y convertirla en lista de palabras
        calle_cliente = row['json']['calle_cliente']
        palabras_calle_cliente = calle_cliente.split()
        palabras_calle_cliente = separar_puntuacion(palabras_calle_cliente)
        
        # Paso 1.3: Extraer el código postal del cliente y convertirlo en lista de palabras
        cp_cliente = row['json']['cp_cliente']
        palabras_cp_cliente = cp_cliente.split()
        palabras_cp_cliente = separar_puntuacion(palabras_cp_cliente)
        
        # Paso 1.4: Extraer la población del cliente y convertirla en lista de palabras
        población_cliente = row['json']['población_cliente']
        palabras_población_cliente = población_cliente.split()
        palabras_población_cliente = separar_puntuacion(palabras_población_cliente)

        # Paso 1.5: Extraer la provincia del cliente y convertirla en lista de palabras
        provincia_cliente = row['json']['provincia_cliente']
        palabras_provincia_cliente = provincia_cliente.split()
        palabras_provincia_cliente = separar_puntuacion(palabras_provincia_cliente)

        # Paso 1.6: Extraer el CIF de la comercializadora y convertirlo en lista de palabras
        cif_comercializadora = row['json']['cif_comercializadora']
        palabras_cif_comercializadora = cif_comercializadora.split()
        palabras_cif_comercializadora = separar_puntuacion(palabras_cif_comercializadora)
        
        # Paso 1.7: Extraer la dirección de la comercializadora y convertirla en lista de palabras
        dirección_comercializadora = row['json']['dirección_comercializadora']
        palabras_dirección_comercializadora = dirección_comercializadora.split()
        palabras_dirección_comercializadora = separar_puntuacion(palabras_dirección_comercializadora)
        
        # Paso 1.8: Extraer el código postal de la comercializadora y convertirlo en lista de palabras
        cp_comercializadora = row['json']['cp_comercializadora']
        palabras_cp_comercializadora = cp_comercializadora.split()
        palabras_cp_comercializadora = separar_puntuacion(palabras_cp_comercializadora)
        
        # Paso 1.9: Extraer la población de la comercializadora y convertirla en lista de palabras
        población_comercializadora = row['json']['población_comercializadora']
        palabras_población_comercializadora = población_comercializadora.split()
        palabras_población_comercializadora = separar_puntuacion(palabras_población_comercializadora)

        # Paso 1.10: Extraer la provincia de la comercializadora y convertirla en lista de palabras
        provincia_comercializadora = row['json']['provincia_comercializadora']
        palabras_provincia_comercializadora = provincia_comercializadora.split()
        palabras_provincia_comercializadora = separar_puntuacion(palabras_provincia_comercializadora)
        
        # Paso 1.11: Extraer el número de factura y convertirlo en lista de palabras
        número_factura = row['json']['número_factura']
        palabras_número_factura = número_factura.split()
        palabras_número_factura = separar_puntuacion(palabras_número_factura)

        # Establecer la localización a español
        locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
        

        #Paso 1.12: Extraer el inicio de periodo y convertirlo en lista de palabras

        # Extraer el inicio_periodo y convertirlo en lista de palabras
        inicio_periodo = row['json']['inicio_periodo']
        # Convertir inicio_periodo a formato datetime
        fecha_inicio = datetime.strptime(inicio_periodo, '%d.%m.%Y')
        
        # Formatear la fecha en diferentes estilos
        fecha_formato_1 = fecha_inicio.strftime('%d de %B de %Y')  # Ejemplo: 26 de agosto de 2018
        fecha_formato_2 = fecha_inicio.strftime('%d/%m/%Y')        # Ejemplo: 26/08/2018
        fecha_formato_3 = fecha_inicio.strftime('%d-%m-%Y')        # Ejemplo: 26-08-2018
        fecha_formato_4 = fecha_inicio.strftime('%d.%m.%Y')        # Ejemplo: 26.08.2018
        fecha_formato_5 = fecha_inicio.strftime('%d-%B-%Y')       # Ejemplo: 26-agosto-2018
        
        # Convertir las fechas formateadas a lista de palabras
        palabras_fecha_formato_1 = separar_puntuacion(fecha_formato_1.split())
        palabras_fecha_formato_2 = separar_puntuacion(fecha_formato_2.split())
        palabras_fecha_formato_3 = separar_puntuacion(fecha_formato_3.split())
        palabras_fecha_formato_4 = separar_puntuacion(fecha_formato_4.split())
        palabras_fecha_formato_5 = separar_puntuacion(fecha_formato_5.split())


         #Paso 1.13: Extraer el fin de periodo y convertirlo en lista de palabras

        # Extraer el fin_periodo y convertirlo en lista de palabras
        fin_periodo = row['json']['fin_periodo']
        # Convertir fin_periodo a formato datetime
        fecha_fin = datetime.strptime(fin_periodo, '%d.%m.%Y')
        
        # Formatear la fecha en diferentes estilos
        fecha_fin_formato_1 = fecha_fin.strftime('%d de %B de %Y')  # Ejemplo: 26 de agosto de 2018
        fecha_fin_formato_2 = fecha_fin.strftime('%d/%m/%Y')        # Ejemplo: 26/08/2018
        fecha_fin_formato_3 = fecha_fin.strftime('%d-%m-%Y')        # Ejemplo: 26-08-2018
        fecha_fin_formato_4 = fecha_fin.strftime('%d.%m.%Y')        # Ejemplo: 26.08.2018
        fecha_fin_formato_5 = fecha_fin.strftime('%d-%B-%Y')        # Ejemplo: 26-agosto-2018
        
        # Convertir las fechas formateadas a lista de palabras
        palabras_fecha_fin_formato_1 = separar_puntuacion(fecha_fin_formato_1.split())
        palabras_fecha_fin_formato_2 = separar_puntuacion(fecha_fin_formato_2.split())
        palabras_fecha_fin_formato_3 = separar_puntuacion(fecha_fin_formato_3.split())
        palabras_fecha_fin_formato_4 = separar_puntuacion(fecha_fin_formato_4.split())
        palabras_fecha_fin_formato_5 = separar_puntuacion(fecha_fin_formato_5.split())

        # Paso 1.14: Extraer el importe de factura y convertirlo en lista de palabras
        importe_factura = str(row['json']['importe_factura'])
        palabras_importe_factura= importe_factura.split()
        palabras_importe_factura = separar_puntuacion(palabras_importe_factura)
        # Agregar la unidad de medida "kWh"
        palabras_importe_factura.append('€')
        

        #Paso 1.14: Extraer la fecha de cargo y convertirlo en lista de palabras

                # Paso 1.13: Extraer el fecha_cargo y convertirlo en lista de palabras
        
        # Extraer el fecha_cargo y convertirlo en lista de palabras
        fecha_cargo = row['json']['fecha_cargo']
        # Convertir fecha_cargo a formato datetime
        fecha_cargo_obj = datetime.strptime(fecha_cargo, '%d.%m.%Y')
        
        # Formatear la fecha en diferentes estilos
        fecha_cargo_formato_1 = fecha_cargo_obj.strftime('%d de %B de %Y')  # Ejemplo: 26 de agosto de 2018
        fecha_cargo_formato_2 = fecha_cargo_obj.strftime('%d/%m/%Y')        # Ejemplo: 26/08/2018
        fecha_cargo_formato_3 = fecha_cargo_obj.strftime('%d-%m-%Y')        # Ejemplo: 26-08-2018
        fecha_cargo_formato_4 = fecha_cargo_obj.strftime('%d.%m.%Y')        # Ejemplo: 26.08.2018
        fecha_cargo_formato_5 = fecha_cargo_obj.strftime('%d-%B-%Y')        # Ejemplo: 26-agosto-2018
        
        # Convertir las fechas formateadas a lista de palabras
        palabras_fecha_cargo_formato_1 = separar_puntuacion(fecha_cargo_formato_1.split())
        palabras_fecha_cargo_formato_2 = separar_puntuacion(fecha_cargo_formato_2.split())
        palabras_fecha_cargo_formato_3 = separar_puntuacion(fecha_cargo_formato_3.split())
        palabras_fecha_cargo_formato_4 = separar_puntuacion(fecha_cargo_formato_4.split())
        palabras_fecha_cargo_formato_5 = separar_puntuacion(fecha_cargo_formato_5.split())

  
        # Paso 1.16: Extraer el consumo del período y convertirlo en lista de palabras
        consumo_periodo = str(row['json']['consumo_periodo'])
        palabras_consumo_periodo = consumo_periodo.split()
        palabras_consumo_periodo = separar_puntuacion(palabras_consumo_periodo)
        # Agregar la unidad de medida "kWh"
        palabras_consumo_periodo.append("kWh")

        
        # Paso 1.16: Extraer la potencia contratada y convertirla en lista de palabras
        potencia_contratada = row['json']['potencia_contratada']
        
        # Reemplazar la coma por el punto decimal y convertir a float
        potencia_contratada_float = float(potencia_contratada.replace(',', '.'))
        
        # Realizar los cálculos para obtener las versiones redondeada y truncada
        potencia_contratada_redondeada_float = round(potencia_contratada_float, 2)
        potencia_contratada_truncada_float = int(potencia_contratada_float * 100) / 100.0
        
        
        # Convertir los resultados nuevamente a cadena y reemplazar el punto por la coma
        potencia_contratada_redondeada = str(potencia_contratada_redondeada_float).replace('.', ',')
        potencia_contratada_truncada = str(potencia_contratada_truncada_float).replace('.', ',')
        
        # Convertir las potencias contratadas a listas de palabras
        palabras_potencia_contratada = separar_puntuacion(potencia_contratada.split())
        palabras_potencia_contratada_redondeada = separar_puntuacion(potencia_contratada_redondeada.split())
        palabras_potencia_contratada_truncada = separar_puntuacion(potencia_contratada_truncada.split())
        
        # Agregar la unidad de medida "kW"
        palabras_potencia_contratada.append("kW")
        palabras_potencia_contratada_redondeada.append("kW")
        palabras_potencia_contratada_truncada.append("kW")

        # Separa puntuacion
        palabras_potencia_contratada = separar_puntuacion(palabras_potencia_contratada)
        palabras_potencia_contratada_redondeada = separar_puntuacion(palabras_potencia_contratada_redondeada)
        palabras_potencia_contratada_truncada = separar_puntuacion(palabras_potencia_contratada_truncada)
        
        # Paso 1.5: Extraer el nombre de la comercializadora y convertirlo en lista de palabras
        nombre_comercializadora = row['json']['nombre_comercializadora']
        palabras_nombre_comercializadora = nombre_comercializadora.split()
        palabras_nombre_comercializadora = separar_puntuacion(palabras_nombre_comercializadora)

        # Paso 2: Buscar la secuencia de palabras en la lista de palabras de la columna 'texto'
        palabras_factura = row['texto']
        for i in range(len(palabras_factura)):
            # Para nombre_cliente
            if palabras_factura[i:i+len(palabras_nombre_cliente)] == palabras_nombre_cliente:
                # Paso 3: Actualizar las etiquetas correspondientes para nombre_cliente
                for j in range(len(palabras_nombre_cliente)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-NOM'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-NOM'
            
            # Para dni_cliente
            if palabras_factura[i:i+len(palabras_dni_cliente)] == palabras_dni_cliente:
                # Paso 3: Actualizar las etiquetas correspondientes para dni_cliente
                for j in range(len(palabras_dni_cliente)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-DNI'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-DNI'
            
            # Para calle_cliente
            if palabras_factura[i:i+len(palabras_calle_cliente)] == palabras_calle_cliente:
                # Paso 3: Actualizar las etiquetas correspondientes para calle_cliente
                for j in range(len(palabras_calle_cliente)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-CAL'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-CAL'
            
            # Para cp_cliente
            if palabras_factura[i:i+len(palabras_cp_cliente)] == palabras_cp_cliente:
                # Paso 3: Actualizar las etiquetas correspondientes para cp_cliente
                for j in range(len(palabras_cp_cliente)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-CP'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-CP'
            
            # Para población_cliente
            if palabras_factura[i:i+len(palabras_población_cliente)] == palabras_población_cliente:
                # Paso 3: Actualizar las etiquetas correspondientes para población_cliente
                for j in range(len(palabras_población_cliente)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-LOC'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-LOC'

            # Para provincia_cliente
            if palabras_factura[i:i+len(palabras_provincia_cliente)] == palabras_provincia_cliente:
                # Paso 3: Actualizar las etiquetas correspondientes para provincia_cliente
                for j in range(len(palabras_provincia_cliente)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-PRO'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-PRO'
            
            # Para cif_comercializadora
            if palabras_factura[i:i+len(palabras_cif_comercializadora)] == palabras_cif_comercializadora:
                # Paso 3: Actualizar las etiquetas correspondientes para cin comercializadora
                for j in range(len(palabras_cif_comercializadora)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-CIF'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-CIF'

            # Para dirección_comercializadora
            if palabras_factura[i:i+len(palabras_dirección_comercializadora)] == palabras_dirección_comercializadora:
                # Paso 3: Actualizar las etiquetas correspondientes para dirección_comercializadora
                for j in range(len(palabras_dirección_comercializadora)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-DIRC'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-DIRC'
            
            # Para cp_comercializadora
            if palabras_factura[i:i+len(palabras_cp_comercializadora)] == palabras_cp_comercializadora:
                # Paso 3: Actualizar las etiquetas correspondientes para cp_comercializadora
                for j in range(len(palabras_cp_comercializadora)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-CPC'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-CPC'
            
            # Para población_comercializadora
            if palabras_factura[i:i+len(palabras_población_comercializadora)] == palabras_población_comercializadora:
                # Paso 3: Actualizar las etiquetas correspondientes para población_comercializadora
                for j in range(len(palabras_población_comercializadora)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-LOCC'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-LOCC'

            # Para provincia_comercializadora
            if palabras_factura[i:i+len(palabras_provincia_comercializadora)] == palabras_provincia_comercializadora:
                # Paso 3: Actualizar las etiquetas correspondientes para población_comercializadora
                for j in range(len(palabras_provincia_comercializadora)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-PROC'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-PROC'
            
            # Para número_factura
            if palabras_factura[i:i+len(palabras_número_factura)] == palabras_número_factura:
                # Paso 3: Actualizar las etiquetas correspondientes para número_factura
                for j in range(len(palabras_número_factura)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-NUMF'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-NUMF'


            # Lista de todas las posibles palabras de formato de fecha
        formatos_de_fecha = [
            palabras_fecha_formato_1,
            palabras_fecha_formato_2,
            palabras_fecha_formato_3,
            palabras_fecha_formato_4,
            palabras_fecha_formato_5
        ]

        # Iterar sobre el texto y las etiquetas en la fila actual
        palabras_factura = row['texto']
        for i in range(len(palabras_factura)):
            for palabras_fecha_formato in formatos_de_fecha:
                # Verificar si alguna de las secuencias de palabras coincide con las formas de la fecha
                if palabras_factura[i:i+len(palabras_fecha_formato)] == palabras_fecha_formato:
                    # Actualizar las etiquetas correspondientes para cada formato de fecha
                    for j in range(len(palabras_fecha_formato)):
                        if i+j < len(palabras_factura):
                            if j == 0:
                                df.at[index, 'ner_tags'][i+j] = 'B-INI'
                            else:
                                df.at[index, 'ner_tags'][i+j] = 'I-INI'


        # Lista de todas las posibles palabras de formato de fecha
        formatos_de_fin_fecha = [
            palabras_fecha_fin_formato_1,
            palabras_fecha_fin_formato_2,
            palabras_fecha_fin_formato_3,
            palabras_fecha_fin_formato_4,
            palabras_fecha_fin_formato_5
        ]

        # Iterar sobre el texto y las etiquetas en la fila actual
        palabras_factura = row['texto']
        for i in range(len(palabras_factura)):
            for palabras_fecha_fin_formato in formatos_de_fin_fecha:
                # Verificar si alguna de las secuencias de palabras coincide con las formas de la fecha
                if palabras_factura[i:i+len(palabras_fecha_fin_formato)] == palabras_fecha_fin_formato:
                    # Actualizar las etiquetas correspondientes para cada formato de fecha
                    for j in range(len(palabras_fecha_fin_formato)):
                        if i+j < len(palabras_factura):
                            if j == 0:
                                df.at[index, 'ner_tags'][i+j] = 'B-FIN'
                            else:
                                df.at[index, 'ner_tags'][i+j] = 'I-FIN'

        # Para importe_factura
            if palabras_factura[i:i+len(palabras_importe_factura)] == palabras_importe_factura:
                # Paso 3: Actualizar las etiquetas correspondientes para importe factura
                for j in range(len(palabras_importe_factura)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-FAC'
                    elif j < len(palabras_importe_factura):
                        df.at[index, 'ner_tags'][i+j] = 'I-FAC'
        
        
                    # Lista de todas las posibles palabras de formato de fecha
        formatos_de_fecha_cargo = [
            palabras_fecha_cargo_formato_1,
            palabras_fecha_cargo_formato_2,
            palabras_fecha_cargo_formato_3,
            palabras_fecha_cargo_formato_4,
            palabras_fecha_cargo_formato_5
        ]
        
        # Iterar sobre el texto y las etiquetas en la fila actual
        palabras_factura = row['texto']
        for i in range(len(palabras_factura)):
            for palabras_fecha_cargo_formato in formatos_de_fecha_cargo:
                # Verificar si alguna de las secuencias de palabras coincide con las formas de la fecha
                if palabras_factura[i:i+len(palabras_fecha_cargo_formato)] == palabras_fecha_cargo_formato:
                    # Actualizar las etiquetas correspondientes para cada formato de fecha
                    for j in range(len(palabras_fecha_cargo_formato)):
                        if i+j < len(palabras_factura):
                            if j == 0:
                                df.at[index, 'ner_tags'][i+j] = 'B-CAR'
                            else:
                                df.at[index, 'ner_tags'][i+j] = 'I-CAR'
            
            # Para consumo_periodo
            if palabras_factura[i:i+len(palabras_consumo_periodo)] == palabras_consumo_periodo:
                # Paso 3: Actualizar las etiquetas correspondientes para consumo_periodo
                for j in range(len(palabras_consumo_periodo)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-PER'
                    elif j < len(palabras_consumo_periodo):
                        df.at[index, 'ner_tags'][i+j] = 'I-PER'
        
            # Para potencia_contratada
            if palabras_factura[i:i+len(palabras_potencia_contratada)] == palabras_potencia_contratada:
                # Paso 3: Actualizar las etiquetas correspondientes para potencia contratada
                for j in range(len(palabras_potencia_contratada)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-POT'
                    elif j < len(palabras_potencia_contratada):
                        df.at[index, 'ner_tags'][i+j] = 'I-POT'

            # Para potencia_contratada redondeada
            if palabras_factura[i:i+len(palabras_potencia_contratada_redondeada)] == palabras_potencia_contratada_redondeada:
                # Paso 3: Actualizar las etiquetas correspondientes para potencia contratada
                for j in range(len(palabras_potencia_contratada_redondeada)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-POT'
                    elif j < len(palabras_potencia_contratada_truncada):
                        df.at[index, 'ner_tags'][i+j] = 'I-POT'

            # Para potencia_contratada truncada
            if palabras_factura[i:i+len(palabras_potencia_contratada_truncada)] == palabras_potencia_contratada_truncada:
                # Paso 3: Actualizar las etiquetas correspondientes para potencia contratada
                for j in range(len(palabras_potencia_contratada_truncada)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-POT'
                    elif j < len(palabras_potencia_contratada_truncada):
                        df.at[index, 'ner_tags'][i+j] = 'I-POT'
        
            

            

            # Para nombre_comercializadora
            if nombre_comercializadora[i:i+len(palabras_nombre_comercializadora)] == palabras_nombre_comercializadora:
                # Paso 3: Actualizar las etiquetas correspondientes para nombre comercializadora
                for j in range(len(palabras_nombre_comercializadora)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-NOMC'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-NOMC'



            

Esta segunda función corrige algunos errores de la primera (algunos nombres de comercializadoras contenían el nombre de la población, la fecha de fin de contrato era la misma que la de fin de periodo siempre).








In [15]:
def crear_etiquetas_2(df):
    for index, row in df.iterrows():
        
        
        # Paso 1.5: Extraer el nombre de la comercializadora y convertirlo en lista de palabras
        nombre_comercializadora = row['json']['nombre_comercializadora']
        palabras_nombre_comercializadora = nombre_comercializadora.split()
        palabras_nombre_comercializadora = separar_puntuacion(palabras_nombre_comercializadora)

        #Paso 1.13: Extraer el fin de periodo y convertirlo en lista de palabras

        # Extraer el fin_periodo y convertirlo en lista de palabras
        fin_periodo = row['json']['fin_periodo']
        # Convertir fin_periodo a formato datetime
        fecha_fin = datetime.strptime(fin_periodo, '%d.%m.%Y')
        
        # Formatear la fecha en diferentes estilos
        fecha_fin_formato_1 = fecha_fin.strftime('%d de %B de %Y')  # Ejemplo: 26 de agosto de 2018
        fecha_fin_formato_2 = fecha_fin.strftime('%d/%m/%Y')        # Ejemplo: 26/08/2018
        #fecha_fin_formato_3 = fecha_fin.strftime('%d-%m-%Y')        # Ejemplo: 26-08-2018
        fecha_fin_formato_4 = fecha_fin.strftime('%d.%m.%Y')        # Ejemplo: 26.08.2018
        #fecha_fin_formato_5 = fecha_fin.strftime('%d-%B-%Y')        # Ejemplo: 26-agosto-2018
        
        # Convertir las fechas formateadas a lista de palabras
        palabras_fecha_fin_formato_1 = separar_puntuacion(fecha_fin_formato_1.split())
        palabras_fecha_fin_formato_2 = separar_puntuacion(fecha_fin_formato_2.split())
        #palabras_fecha_fin_formato_3 = separar_puntuacion(fecha_fin_formato_3.split())
        palabras_fecha_fin_formato_4 = separar_puntuacion(fecha_fin_formato_4.split())
        #palabras_fecha_fin_formato_5 = separar_puntuacion(fecha_fin_formato_5.split())

        

        # Añadir secuencia para identificar final de contrato

        secuencia_1 = ['(', 'renovación', 'anual', 'automática', ')']
        secuencia_2 = ['Datos', 'instalación', 'electricidad']
        
        
        palabras_fecha_fin_formato_11 = palabras_fecha_fin_formato_1 + secuencia_1
        palabras_fecha_fin_formato_21 = palabras_fecha_fin_formato_2 + secuencia_1
        palabras_fecha_fin_formato_31 = palabras_fecha_fin_formato_4 + secuencia_2

        
        
        
        # Paso 2: Buscar la secuencia de palabras en la lista de palabras de la columna 'texto'
        palabras_factura = row['texto']
        for i in range(len(palabras_factura)):
        
            # Para nombre_comercializadora
            if palabras_factura[i:i+len(palabras_nombre_comercializadora)] == palabras_nombre_comercializadora:
                # Paso 3: Actualizar las etiquetas correspondientes para nombre_comercializadora
                for j in range(len(palabras_nombre_comercializadora)):
                    if j == 0:
                        df.at[index, 'ner_tags'][i+j] = 'B-NOMC'
                    else:
                        df.at[index, 'ner_tags'][i+j] = 'I-NOMC'

# Lista de todas las posibles palabras de formato de fecha
        formatos_de_fin_fecha = [
            palabras_fecha_fin_formato_11,
            palabras_fecha_fin_formato_21,
            palabras_fecha_fin_formato_31
        ]

        # Iterar sobre el texto y las etiquetas en la fila actual
        palabras_factura = row['texto']
        for i in range(len(palabras_factura)):
            for palabras_fecha_fin_formato in formatos_de_fin_fecha:
                # Verificar si alguna de las secuencias de palabras coincide con las formas de la fecha
                if palabras_factura[i:i+len(palabras_fecha_fin_formato)] == palabras_fecha_fin_formato:
                    # Actualizar las etiquetas correspondientes para cada formato de fecha
                    for j in range(len(palabras_fecha_fin_formato)):
                        if i+j < len(palabras_factura):
                            if j == 0:
                                df.at[index, 'ner_tags'][i+j] = 'O'
                            else:
                                df.at[index, 'ner_tags'][i+j] = 'O'


Esta función recorre el camino inverso para comprobar que el etiquetado se ha realizado correctamente. Busca las etiquetas en 'ner_tags', con sus índices busca la clave en 'texto' y la guarda en un diccionario en 'json_comprobación'.








In [16]:
def comprobar_etiquetas(df):
    for index, row in df.iterrows():
        # Inicializar listas para almacenar las ocurrencias de las diferentes entidades como cadena
        ocurrencias_nombre_cliente = []
        ocurrencias_dni_cliente = []
        ocurrencias_calle_cliente = []
        ocurrencias_cp_cliente = []
        ocurrencias_población_cliente = []
        ocurrencias_provincia_cliente = []  
        ocurrencias_nombre_comercializadora = []
        ocurrencias_cif_comercializadora = []
        ocurrencias_dirección_comercializadora = []
        ocurrencias_cp_comercializadora = []
        ocurrencias_población_comercializadora = []
        ocurrencias_provincia_comercializadora = []  
        ocurrencias_número_factura = []
        ocurrencias_inicio_periodo = []
        ocurrencias_fin_periodo = []
        ocurrencias_importe_factura = []
        ocurrencias_fecha_cargo = []
        ocurrencias_consumo_periodo = []
        ocurrencias_potencia_contratada = []
        
        # Inicializar listas para almacenar las palabras de las diferentes entidades
        nombre_cliente = []
        dni_cliente = []
        calle_cliente = []
        cp_cliente = []
        población_cliente = []
        provincia_cliente = []  
        nombre_comercializadora = []
        cif_comercializadora = []
        dirección_comercializadora = []
        cp_comercializadora = []
        población_comercializadora = []
        provincia_comercializadora = [] 
        número_factura = []
        inicio_periodo = []
        fin_periodo = []
        importe_factura = []
        fecha_cargo = []
        consumo_periodo = []
        potencia_contratada = []
        
        # Iterar sobre las etiquetas para buscar secuencias correspondientes a diferentes entidades
        for i, etiqueta in enumerate(row['ner_tags']):
            if etiqueta == 'B-NOM':
                # Reiniciar la lista de palabras del nombre del cliente para cada nueva ocurrencia
                nombre_cliente = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-NOM' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-NOM':
                        # Agregar la palabra al nombre del cliente
                        nombre_cliente.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NOM'
                # Agregar la ocurrencia del nombre del cliente como cadena a la lista
                ocurrencias_nombre_cliente.append(' '.join(unir_puntuacion(nombre_cliente)))
            
            elif etiqueta == 'B-DNI':
                # Reiniciar la lista de palabras del DNI del cliente para cada nueva ocurrencia
                dni_cliente = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-DNI' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-DNI':
                        # Agregar la palabra al DNI del cliente
                        dni_cliente.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-DNI'
                # Agregar la ocurrencia del DNI del cliente como cadena a la lista
                ocurrencias_dni_cliente.append(' '.join(unir_puntuacion(dni_cliente)))
            
            elif etiqueta == 'B-CAL':
                # Reiniciar la lista de palabras de la calle del cliente para cada nueva ocurrencia
                calle_cliente = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-CAL' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-CAL':
                        # Agregar la palabra a la calle del cliente
                        calle_cliente.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-CAL'
                # Agregar la ocurrencia de la calle del cliente como cadena a la lista
                ocurrencias_calle_cliente.append(' '.join(unir_puntuacion(calle_cliente)))
            
            elif etiqueta == 'B-CP':
                # Reiniciar la lista de palabras del código postal del cliente para cada nueva ocurrencia
                cp_cliente = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-CP' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-CP':
                        # Agregar la palabra al código postal del cliente
                        cp_cliente.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-CP'
                # Agregar la ocurrencia del código postal del cliente como cadena a la lista
                ocurrencias_cp_cliente.append(' '.join(unir_puntuacion(cp_cliente)))
            
            elif etiqueta == 'B-LOC':
                # Reiniciar la lista de palabras de la población del cliente para cada nueva ocurrencia
                población_cliente = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-LOC' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-LOC':
                        # Agregar la palabra a la población del cliente
                        población_cliente.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-LOC'
                # Agregar la ocurrencia de la población del cliente como cadena a la lista
                ocurrencias_población_cliente.append(' '.join(unir_puntuacion(población_cliente)))
                
            elif etiqueta == 'B-PRO':
                provincia_cliente = [row['texto'][i]]
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-PRO':
                        provincia_cliente.append(row['texto'][j])
                    else:
                        break
                ocurrencias_provincia_cliente.append(' '.join(unir_puntuacion(provincia_cliente)))
                
            elif etiqueta == 'B-NOMC':
                # Reiniciar la lista de palabras del nombre de la comercializadora para cada nueva ocurrencia
                nombre_comercializadora = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-NOMC' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-NOMC':
                        # Agregar la palabra al nombre de la comercializadora
                        nombre_comercializadora.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NOMC'
                # Agregar la ocurrencia del nombre de la comercializadora como cadena a la lista
                ocurrencias_nombre_comercializadora.append(' '.join(unir_puntuacion(nombre_comercializadora)))
                
            elif etiqueta == 'B-CIF':
                # Reiniciar la lista de palabras del CIF de la comercializadora para cada nueva ocurrencia
                cif_comercializadora = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-CIF' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-CIF':
                        # Agregar la palabra al CIF de la comercializadora
                        cif_comercializadora.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-CIF'
                # Agregar la ocurrencia del CIF de la comercializadora como cadena a la lista
                ocurrencias_cif_comercializadora.append(' '.join(unir_puntuacion(cif_comercializadora)))
                
            elif etiqueta == 'B-DIRC':
                # Reiniciar la lista de palabras de la dirección de la comercializadora para cada nueva ocurrencia
                dirección_comercializadora = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-DIRC' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-DIRC':
                        # Agregar la palabra a la dirección de la comercializadora
                        dirección_comercializadora.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-DIRC'
                # Agregar la ocurrencia de la dirección de la comercializadora como cadena a la lista
                ocurrencias_dirección_comercializadora.append(' '.join(unir_puntuacion(dirección_comercializadora)))
                
            elif etiqueta == 'B-CPC':
                # Reiniciar la lista de palabras del código postal de la comercializadora para cada nueva ocurrencia
                cp_comercializadora = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-CPC' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-CPC':
                        # Agregar la palabra al código postal de la comercializadora
                        cp_comercializadora.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-CPC'
                # Agregar la ocurrencia del código postal de la comercializadora como cadena a la lista
                ocurrencias_cp_comercializadora.append(' '.join(unir_puntuacion(cp_comercializadora)))
                
            elif etiqueta == 'B-LOCC':
                # Reiniciar la lista de palabras de la población de la comercializadora para cada nueva ocurrencia
                población_comercializadora = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-LOCC' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-LOCC':
                        # Agregar la palabra a la población de la comercializadora
                        población_comercializadora.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-LOCC'
                # Agregar la ocurrencia de la población de la comercializadora como cadena a la lista
                ocurrencias_población_comercializadora.append(' '.join(unir_puntuacion(población_comercializadora)))
                
            elif etiqueta == 'B-PROC':
                provincia_comercializadora = [row['texto'][i]]
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-PROC':
                        provincia_comercializadora.append(row['texto'][j])
                    else:
                        break
                ocurrencias_provincia_comercializadora.append(' '.join(unir_puntuacion(provincia_comercializadora)))
            
            elif etiqueta == 'B-NUMF':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                número_factura = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-NUMF' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-NUMF':
                        # Agregar la palabra al número de factura
                        número_factura.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_número_factura.append(' '.join(unir_puntuacion(número_factura)))

            elif etiqueta == 'B-INI':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                inicio_periodo = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-NUMF' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-INI':
                        # Agregar la palabra al número de factura
                        inicio_periodo.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_inicio_periodo.append(' '.join(unir_puntuacion(inicio_periodo)))

            elif etiqueta == 'B-FIN':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                fin_periodo = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-NUMF' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-FIN':
                        # Agregar la palabra al número de factura
                        fin_periodo.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_fin_periodo.append(' '.join(unir_puntuacion(fin_periodo)))

            elif etiqueta == 'B-FAC':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                importe_factura = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-FAC' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-FAC':
                        # Agregar la palabra al número de factura
                        importe_factura.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_importe_factura.append(' '.join(unir_puntuacion(importe_factura)))

            elif etiqueta == 'B-CAR':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                fecha_cargo = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-NUMF' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-CAR':
                        # Agregar la palabra al número de factura
                        fecha_cargo.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_fecha_cargo.append(' '.join(unir_puntuacion(fecha_cargo)))

            elif etiqueta == 'B-PER':
                # Reiniciar la lista de palabras del número de factura para cada nueva ocurrencia
                consumo_periodo = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-NUMF' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-PER':
                        # Agregar la palabra al número de factura
                        consumo_periodo.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_consumo_periodo.append(' '.join(unir_puntuacion(consumo_periodo)))

            elif etiqueta == 'B-POT':
                # Reiniciar la lista de palabras de potencia_contatada para cada nueva ocurrencia
                potencia_contratada = [row['texto'][i]]
                # Buscar si hay etiquetas 'I-POT' después
                for j in range(i+1, len(row['ner_tags'])):
                    if row['ner_tags'][j] == 'I-POT':
                        # Agregar la palabra al número de factura
                        potencia_contratada.append(row['texto'][j])
                    else:
                        break  # Salir del bucle cuando no haya más etiquetas 'I-NUMF'
                # Agregar la ocurrencia del número de factura como cadena a la lista
                ocurrencias_potencia_contratada.append(' '.join(unir_puntuacion(potencia_contratada)))
                
        # Agregar las ocurrencias al DataFrame bajo la columna 'json_comprobacion'
        df.at[index, 'json_comprobacion']['nombre_cliente'] = ocurrencias_nombre_cliente
        df.at[index, 'json_comprobacion']['dni_cliente'] = ocurrencias_dni_cliente
        df.at[index, 'json_comprobacion']['calle_cliente'] = ocurrencias_calle_cliente
        df.at[index, 'json_comprobacion']['cp_cliente'] = ocurrencias_cp_cliente
        df.at[index, 'json_comprobacion']['población_cliente'] = ocurrencias_población_cliente
        df.at[index, 'json_comprobacion']['provincia_cliente'] = ocurrencias_provincia_cliente  # Agregar provincia_cliente
        df.at[index, 'json_comprobacion']['nombre_comercializadora'] = ocurrencias_nombre_comercializadora
        df.at[index, 'json_comprobacion']['cif_comercializadora'] = ocurrencias_cif_comercializadora
        df.at[index, 'json_comprobacion']['dirección_comercializadora'] = ocurrencias_dirección_comercializadora
        df.at[index, 'json_comprobacion']['cp_comercializadora'] = ocurrencias_cp_comercializadora
        df.at[index, 'json_comprobacion']['población_comercializadora'] = ocurrencias_población_comercializadora
        df.at[index, 'json_comprobacion']['provincia_comercializadora'] = ocurrencias_provincia_comercializadora  # Agregar provincia_comercializadora
        df.at[index, 'json_comprobacion']['número_factura'] = ocurrencias_número_factura
        df.at[index, 'json_comprobacion']['inicio_periodo'] = ocurrencias_inicio_periodo
        df.at[index, 'json_comprobacion']['fin_periodo'] = ocurrencias_fin_periodo
        df.at[index, 'json_comprobacion']['importe_factura'] = ocurrencias_importe_factura
        df.at[index, 'json_comprobacion']['fecha_cargo'] = ocurrencias_fecha_cargo
        df.at[index, 'json_comprobacion']['consumo_periodo'] = ocurrencias_consumo_periodo
        df.at[index, 'json_comprobacion']['potencia_contratada'] = ocurrencias_potencia_contratada
    
    
    return df



In [17]:
crear_etiquetas(df)
crear_etiquetas_2(df)
comprobar_etiquetas(df)

,id,texto,ner_tags,json,original,json_comprobacion
0,factura_836,"[DATOS, DE, LA, FACTURA, IMPORTE, FACTURA, :, ...","[O, O, O, O, O, O, O, B-FAC, I-FAC, O, O, O, B...","{'nombre_cliente': 'ERMINIA CUESTA MACHADO', '...",1,"{'nombre_cliente': ['ERMINIA CUESTA MACHADO', ..."
1,factura_264,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...",1,"{'nombre_cliente': ['SALUD MONTALVO CUÉLLAR', ..."
2,factura_533,"[DATOS, DE, LA, FACTURA, Nº, factura, :, I3360...","[O, O, O, O, O, O, O, B-NUMF, O, O, O, O, O, O...","{'nombre_cliente': 'Iluminada Donato Vallejo',...",1,{'nombre_cliente': ['Iluminada Donato Vallejo'...
3,factura_311,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",{'nombre_cliente': 'Celestino Estrada Santiago...,1,{'nombre_cliente': ['Celestino Estrada Santiag...
4,factura_473,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '...",1,"{'nombre_cliente': ['Candelaria Ayala Icaza', ..."
...,...,...,...,...,...,...
1823,factura_156_sh,"[X,XX, €, DETALLE, DE, LA, FACTURA, 35002, Las...","[O, O, O, O, O, O, B-CP, B-PRO, I-PRO, I-LOC, ...","{'nombre_cliente': 'SATURIO LEYVA TAMES', 'dni...",0,"{'nombre_cliente': ['SATURIO LEYVA TAMES', 'SA..."
1824,factura_97_sh,"[automática, ), porcentajes, del, promedio, na...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Fermín Vívez Padilla', 'dn...",0,"{'nombre_cliente': ['Fermín Vívez Padilla', 'F..."
1825,factura_501_sh,"[DATOS, DE, LA, FACTURA, 14546, Santaella, CIF...","[O, O, O, O, B-CP, B-LOC, O, B-CIF, O, B-DIRC,...","{'nombre_cliente': 'EUSTAQUIO ARROLLO NIETO', ...",0,"{'nombre_cliente': ['EUSTAQUIO ARROLLO NIETO',..."
1826,factura_731_sh,"[Fuente, :, CNMC, (, Comisión, Nacional, de, l...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'CIRIACO LIRA PEÑA', 'dni_c...",0,"{'nombre_cliente': ['CIRIACO LIRA PEÑA', 'CIRI..."


Ya que puede haber varias apariciones de cada clave, esta función selecciona una y realiza una serie de tratamientos para poder comparar las etiquetas extraídas con las reales.








In [18]:
def process_dataframe(df):
    def transform_json(json_data):
        def most_common(lst):
            if not lst:
                return None
            if 'potencia_contratada' in lst:
                # Para la entrada 'potencia_contratada', seleccionar por mayor longitud y en caso de empate, el primero
                potencia_contratada_values = [item for item in lst if isinstance(item, str)]
                max_len_val = max(potencia_contratada_values, key=lambda x: (len(x), lst.index(x)))
                return max_len_val
            else:
                count = Counter(lst)
                most_common_val = count.most_common(1)[0][0]
                # Si hay un empate, seleccionar el de mayor longitud
                max_len_val = max(lst, key=len)
                return max_len_val if lst.count(max_len_val) > 1 else most_common_val
        
        def format_date(date_str):
            formats = [
                "%d de %B de %Y",
                "%d/%m/%Y",
                "%d-%m-%Y",
                "%d.%m.%Y",
                "%d-%B-%Y"
            ]
            for fmt in formats:
                try:
                    date_obj = datetime.strptime(date_str, fmt)
                    return date_obj.strftime("%d.%m.%Y")
                except ValueError:
                    pass
            return date_str

        # Crear el nuevo diccionario con los valores más comunes
        new_json = {key: (most_common(value) if isinstance(value, list) else value) for key, value in json_data.items()}
        
        # Formatear las fechas si están en el diccionario
        for date_key in ['inicio_periodo', 'fin_periodo', 'fecha_cargo']:
            if date_key in new_json:
                new_json[date_key] = format_date(new_json[date_key])
        
        # Convertir consumo_periodo a entero si existe
        if 'consumo_periodo' in new_json:
            try:
                new_json['consumo_periodo'] = int(new_json['consumo_periodo'])
            except ValueError:
                pass  # Manejar el caso en el que la conversión no sea posible
        
        return new_json
    
    # Aplicar la transformación a cada fila y crear la nueva columna json_2
    df['json_2'] = df['json_comprobacion'].apply(transform_json)
    return df


In [19]:
process_dataframe(df)

,id,texto,ner_tags,json,original,json_comprobacion,json_2
0,factura_836,"[DATOS, DE, LA, FACTURA, IMPORTE, FACTURA, :, ...","[O, O, O, O, O, O, O, B-FAC, I-FAC, O, O, O, B...","{'nombre_cliente': 'ERMINIA CUESTA MACHADO', '...",1,"{'nombre_cliente': ['ERMINIA CUESTA MACHADO', ...","{'nombre_cliente': 'ERMINIA CUESTA MACHADO', '..."
1,factura_264,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...",1,"{'nombre_cliente': ['SALUD MONTALVO CUÉLLAR', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '..."
2,factura_533,"[DATOS, DE, LA, FACTURA, Nº, factura, :, I3360...","[O, O, O, O, O, O, O, B-NUMF, O, O, O, O, O, O...","{'nombre_cliente': 'Iluminada Donato Vallejo',...",1,{'nombre_cliente': ['Iluminada Donato Vallejo'...,"{'nombre_cliente': 'Iluminada Donato Vallejo',..."
3,factura_311,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",{'nombre_cliente': 'Celestino Estrada Santiago...,1,{'nombre_cliente': ['Celestino Estrada Santiag...,{'nombre_cliente': 'Celestino Estrada Santiago...
4,factura_473,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '...",1,"{'nombre_cliente': ['Candelaria Ayala Icaza', ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '..."
...,...,...,...,...,...,...,...
1823,factura_156_sh,"[X,XX, €, DETALLE, DE, LA, FACTURA, 35002, Las...","[O, O, O, O, O, O, B-CP, B-PRO, I-PRO, I-LOC, ...","{'nombre_cliente': 'SATURIO LEYVA TAMES', 'dni...",0,"{'nombre_cliente': ['SATURIO LEYVA TAMES', 'SA...","{'nombre_cliente': 'SATURIO LEYVA TAMES', 'dni..."
1824,factura_97_sh,"[automática, ), porcentajes, del, promedio, na...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Fermín Vívez Padilla', 'dn...",0,"{'nombre_cliente': ['Fermín Vívez Padilla', 'F...","{'nombre_cliente': 'Fermín Vívez Padilla', 'dn..."
1825,factura_501_sh,"[DATOS, DE, LA, FACTURA, 14546, Santaella, CIF...","[O, O, O, O, B-CP, B-LOC, O, B-CIF, O, B-DIRC,...","{'nombre_cliente': 'EUSTAQUIO ARROLLO NIETO', ...",0,"{'nombre_cliente': ['EUSTAQUIO ARROLLO NIETO',...","{'nombre_cliente': 'EUSTAQUIO ARROLLO NIETO', ..."
1826,factura_731_sh,"[Fuente, :, CNMC, (, Comisión, Nacional, de, l...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'CIRIACO LIRA PEÑA', 'dni_c...",0,"{'nombre_cliente': ['CIRIACO LIRA PEÑA', 'CIRI...","{'nombre_cliente': 'CIRIACO LIRA PEÑA', 'dni_c..."


In [20]:
df['json_2'][0]

{'nombre_cliente': 'ERMINIA CUESTA MACHADO',
 'dni_cliente': '75723815H',
 'calle_cliente': 'Calle Camelia',
 'cp_cliente': '24900',
 'población_cliente': 'Riaño',
 'provincia_cliente': 'León',
 'nombre_comercializadora': 'ENERGYA VM GENERACIÓN, S.L',
 'cif_comercializadora': 'B84070036',
 'dirección_comercializadora': 'C/ RIBERA DEL LOIRA, Nº 6',
 'cp_comercializadora': '28042',
 'población_comercializadora': 'MADRID',
 'provincia_comercializadora': 'Madrid',
 'número_factura': 'P7071578078',
 'inicio_periodo': '26.06.1991',
 'fin_periodo': '26.07.1991',
 'importe_factura': '21,09 €',
 'fecha_cargo': '31.07.1991',
 'consumo_periodo': '25 kWh',
 'potencia_contratada': '3,55 kW'}

Calculamos el porcentaje de coincidencia para cada clave.

In [21]:
def calcular_porcentaje_promedio_coincidencia(df):
    coincidencias_totales = {}
    total_filas = len(df)
    
    # Iterar sobre cada fila del DataFrame
    for index, row in df.iterrows():
        json_original = row['json']
        json_transformado = row['json_2']
        
        # Comparar cada entrada de los diccionarios json y json_2
        for key, value in json_original.items():
            if key not in coincidencias_totales:
                coincidencias_totales[key] = {'total': 0, 'coincidencias': 0}
            
            # Si la clave está en ambos diccionarios y los valores coinciden, sumar 1 a las coincidencias totales
            coincidencias_totales[key]['total'] += 1
            if key in json_transformado and json_transformado[key] == value:
                coincidencias_totales[key]['coincidencias'] += 1
    
    # Calcular el porcentaje promedio de coincidencia para cada clave
    for key, values in coincidencias_totales.items():
        coincidencias_totales[key] = (values['coincidencias'] / values['total']) * 100 if values['total'] != 0 else 0
    
    return coincidencias_totales

# Uso de la función para calcular el porcentaje promedio de coincidencia entre json y json_2
porcentaje_promedio_coincidencia = calcular_porcentaje_promedio_coincidencia(df)


In [22]:
calcular_porcentaje_promedio_coincidencia(df)

{'nombre_cliente': 99.45295404814004,
 'dni_cliente': 100.0,
 'calle_cliente': 100.0,
 'cp_cliente': 100.0,
 'población_cliente': 94.9671772428884,
 'provincia_cliente': 81.0722100656455,
 'nombre_comercializadora': 95.18599562363238,
 'cif_comercializadora': 99.78118161925602,
 'dirección_comercializadora': 95.40481400437638,
 'cp_comercializadora': 94.09190371991247,
 'población_comercializadora': 64.87964989059081,
 'provincia_comercializadora': 99.78118161925602,
 'número_factura': 100.0,
 'inicio_periodo': 100.0,
 'fin_periodo': 100.0,
 'importe_factura': 0.0,
 'fecha_cargo': 100.0,
 'consumo_periodo': 0.0,
 'potencia_contratada': 0.0}

Los resultados en general son buenos. El 0% de importe_factura, fecha_cargo y potencia_contratada se deben a que han sido etiquetados junto a "€", "kWh" y "kW", para mejorar el rendimiento del modelo.








In [23]:
#Ruta para guardar el archivo CSV
csv_path = "Data/dataset_facturas_aumentado_etiquetado.csv"

# Guardar el DataFrame como un archivo CSV
df.to_csv(csv_path, index=False)#

print(f"El dataset se ha guardado en {csv_path}")

El dataset se ha guardado en Data/dataset_facturas_aumentado_etiquetado.csv


En una primera fase del entrenamiento del modelo, vamos a separar uno de los tipos de factura (el más distinto del resto), para hacernos una idea de qué tan bien generaliza el modelo. Como esta porción no se va a usar para el entrenamiento, le quitamos también las filas aumentadas.








In [24]:
# Definir la secuencia de palabras que queremos buscar en tokens
sequence_to_find = ['Ideas', 'que', 'facilitan', 'su', 'día']

# Función para encontrar la secuencia en la lista de tokens
def find_sequence(texto):
    for i in range(len(texto) - len(sequence_to_find) + 1):
        if texto[i:i+len(sequence_to_find)] == sequence_to_find:
            return True
    return False

# Filtrar las filas que contienen la secuencia en 'tokens'
test_rows = df[df['texto'].apply(find_sequence)]

# Crear el DataFrame test con las filas seleccionadas
test = pd.DataFrame(test_rows)

# Eliminar las filas seleccionadas del DataFrame original
df.drop(test_rows.index, inplace=True)

# Verificar el nuevo DataFrame de prueba
test.head(3)

,id,texto,ner_tags,json,original,json_comprobacion,json_2
1,factura_264,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...",1,"{'nombre_cliente': ['SALUD MONTALVO CUÉLLAR', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '..."
3,factura_311,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",{'nombre_cliente': 'Celestino Estrada Santiago...,1,{'nombre_cliente': ['Celestino Estrada Santiag...,{'nombre_cliente': 'Celestino Estrada Santiago...
4,factura_473,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '...",1,"{'nombre_cliente': ['Candelaria Ayala Icaza', ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '..."


In [25]:
test = test[test['original'] != 0]

In [26]:
test

,id,texto,ner_tags,json,original,json_comprobacion,json_2
1,factura_264,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...",1,"{'nombre_cliente': ['SALUD MONTALVO CUÉLLAR', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '..."
3,factura_311,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",{'nombre_cliente': 'Celestino Estrada Santiago...,1,{'nombre_cliente': ['Celestino Estrada Santiag...,{'nombre_cliente': 'Celestino Estrada Santiago...
4,factura_473,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '...",1,"{'nombre_cliente': ['Candelaria Ayala Icaza', ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '..."
7,factura_319,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Oliverio Peraza Juárez', '...",1,"{'nombre_cliente': ['Oliverio Peraza Juárez', ...","{'nombre_cliente': 'Oliverio Peraza Juárez', '..."
9,factura_283,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Orangel Salgado Chavira', ...",1,"{'nombre_cliente': ['Orangel Salgado Chavira',...","{'nombre_cliente': 'Orangel Salgado Chavira', ..."
...,...,...,...,...,...,...,...
891,factura_545,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Victorio Cuervo Ferrer', '...",1,"{'nombre_cliente': ['Victorio Cuervo Ferrer', ...","{'nombre_cliente': 'Victorio Cuervo Ferrer', '..."
894,factura_761,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'LUCIA CARVALLO NOBLE', 'dn...",1,"{'nombre_cliente': ['LUCIA CARVALLO NOBLE', 'L...","{'nombre_cliente': 'LUCIA CARVALLO NOBLE', 'dn..."
896,factura_114,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Simón Gamboa Ayala', 'dni_...",1,"{'nombre_cliente': ['Simón Gamboa Ayala', 'Sim...","{'nombre_cliente': 'Simón Gamboa Ayala', 'dni_..."
900,factura_418,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",{'nombre_cliente': 'CONSTANZA SARDINAS NAVARO'...,1,{'nombre_cliente': ['CONSTANZA SARDINAS NAVARO...,{'nombre_cliente': 'CONSTANZA SARDINAS NAVARO'...


In [27]:
combined_df = pd.concat([test, df], ignore_index=True)

In [28]:
combined_df

,id,texto,ner_tags,json,original,json_comprobacion,json_2
0,factura_264,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '...",1,"{'nombre_cliente': ['SALUD MONTALVO CUÉLLAR', ...","{'nombre_cliente': 'SALUD MONTALVO CUÉLLAR', '..."
1,factura_311,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",{'nombre_cliente': 'Celestino Estrada Santiago...,1,{'nombre_cliente': ['Celestino Estrada Santiag...,{'nombre_cliente': 'Celestino Estrada Santiago...
2,factura_473,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '...",1,"{'nombre_cliente': ['Candelaria Ayala Icaza', ...","{'nombre_cliente': 'Candelaria Ayala Icaza', '..."
3,factura_319,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Oliverio Peraza Juárez', '...",1,"{'nombre_cliente': ['Oliverio Peraza Juárez', ...","{'nombre_cliente': 'Oliverio Peraza Juárez', '..."
4,factura_283,"[L, .., ., u, ., ., n, ..e, ., ., s, ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Orangel Salgado Chavira', ...",1,"{'nombre_cliente': ['Orangel Salgado Chavira',...","{'nombre_cliente': 'Orangel Salgado Chavira', ..."
...,...,...,...,...,...,...,...
1548,factura_927_sh,[................................................,"[O, O, O, O, O, O, O, B-INI, O, B-FIN, O, O, O...",{'nombre_cliente': 'MAGIA ECHEVERRÍA SALDIVAR'...,0,{'nombre_cliente': ['MAGIA ECHEVERRÍA SALDIVAR...,{'nombre_cliente': 'MAGIA ECHEVERRÍA SALDIVAR'...
1549,factura_156_sh,"[X,XX, €, DETALLE, DE, LA, FACTURA, 35002, Las...","[O, O, O, O, O, O, B-CP, B-PRO, I-PRO, I-LOC, ...","{'nombre_cliente': 'SATURIO LEYVA TAMES', 'dni...",0,"{'nombre_cliente': ['SATURIO LEYVA TAMES', 'SA...","{'nombre_cliente': 'SATURIO LEYVA TAMES', 'dni..."
1550,factura_97_sh,"[automática, ), porcentajes, del, promedio, na...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","{'nombre_cliente': 'Fermín Vívez Padilla', 'dn...",0,"{'nombre_cliente': ['Fermín Vívez Padilla', 'F...","{'nombre_cliente': 'Fermín Vívez Padilla', 'dn..."
1551,factura_501_sh,"[DATOS, DE, LA, FACTURA, 14546, Santaella, CIF...","[O, O, O, O, B-CP, B-LOC, O, B-CIF, O, B-DIRC,...","{'nombre_cliente': 'EUSTAQUIO ARROLLO NIETO', ...",0,"{'nombre_cliente': ['EUSTAQUIO ARROLLO NIETO',...","{'nombre_cliente': 'EUSTAQUIO ARROLLO NIETO', ..."


In [29]:
#Ruta para guardar el archivo CSV
csv_path = "Data/dataset_facturas_aumentado_etiquetado_test.csv"

# Guardar el DataFrame como un archivo CSV
combined_df.to_csv(csv_path, index=False)#

print(f"El dataset se ha guardado en {csv_path}")

El dataset se ha guardado en Data/dataset_facturas_aumentado_etiquetado_test.csv
